# Recommender Systems
Using Popularity Ranking, User-Based Collaborative Filtering and Item-Based Collaborative Filtering to recommend movies based on user or item inputs. Recommenders scored using various metrics on an offline train-test split

## Imports

In [ ]:
# Importing Libraries
import pandas as pd
from ydata_profiling import ProfileReport
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import r2_score, mean_absolute_error

In [ ]:
# Reading Data
links_df = pd.read_csv('../Data/links.csv')
movies_df = pd.read_csv('../Data/movies.csv')
ratings_df = pd.read_csv('../Data/ratings.csv')
tags_df =pd.read_csv('../Data/tags.csv')

## Data Exploration

In [ ]:
# Creating y-data profiling reports as html file
movies_profile = ProfileReport(movies_df, minimal=True).to_file('movies_report.html')
ratings_profile = ProfileReport(ratings_df, minimal=True).to_file('ratings_report.html')

# Recommenders

## Popularity Ranking

In [ ]:
# Creating Column to Track Review Count During Aggregations
ratings_df['rating_count'] = 1

### Average Ranking, Filtered with Review Minimum

### Laplace Inspired Data Manipulation

### Cumulative Rating

## Collaborative Filtering

### Item-Based Collaborative Filtering

### User-Based Collaborative Filtering

## Evaluating Recommenders with Offline Methods